# Minimal OAuth Authorization Code Flow (Pure Python)

This notebook simulates the OAuth 2.0 Authorization Code flow
using pure Python functions.

Goals:
- Understand execution order
- See where invariants are enforced
- Avoid framework abstractions


## Actors (Conceptual Mapping)

- Client (Agent): initiates flow
- Authorization Server: validates and issues tokens
- Resource Server: validates tokens only

In this notebook, each actor is represented
by a Python function.


In [7]:
# authorization server state (represents server side memory)
REGISTERED_REDIRECT_URIS = {
    "client-123": ["https://agent.example.com/callback"]
}

AUTHORIZATION_CODES = {}
ISSUED_TOKENS = {}

In [2]:
print(REGISTERED_REDIRECT_URIS.get("client-123", []))

['https://agent.example.com/callback']


In [9]:
# utility function to enforce https
def require_https(url: str):
    assert url.startswith("https://"), "URL must be https"

def require_exact_uri(client_id: str, redirect_uri: str):
    allowed = REGISTERED_REDIRECT_URIS.get(client_id, [])
    assert redirect_uri in allowed, "redirect_uri mismatched"

In [ ]:
# AUTHORIZATION_CODES["code"] = {
#     "client_id" : "example_client_id",
#     "redirect_uri" : "example_redirect_uri",
#     "code_challenge" : "example_code_challenge",
#     "scope" : "example_scope"
# # }
# import json

# print(json.dumps(AUTHORIZATION_CODES, indent=2) )

{
  "code": {
    "client_id": "example_client_id",
    "redirect_uri": "example_redirect_uri",
    "code_challenge": "example_code_challenge",
    "scope": "example_scope"
  }
}


In [4]:
# Client → Authorization Server (/authorize) : Authorization Request (Code)

import secrets

def authorize_request(
    client_id: str,
    redirect_uri: str,
    scope: str,
    code_challenge: str
):
    # Invariants enforced here
    require_https(redirect_uri)
    require_exact_uri(client_id, redirect_uri)
    assert code_challenge, "PKCE required"
    
    # Issue authorization code
    code = secrets.token_urlsafe(16)
    
    AUTHORIZATION_CODES[code] = {
        "client_id" : client_id,
        "redirect_uri" : redirect_uri,
        "code_challenge" : code_challenge,
        "scope" : scope
    }
    
    return code

In [ ]:
# Client → Authorization Server (/token) : Token Exchange (Code)

import hashlib
import base64

def s256(verifier: str)-> str:
    digest = hashlib.sha256(verifier.encode()).digest()
    return base64.urlsafe_b64encode(digest).rstrip(b'=').decode('ascii')


def token_request(
    code: str,
    client_id: str,
    redirect_uri: str,
    code_verifier: str
):
    record = AUTHORIZATION_CODES.pop(code, None)
    assert record, "invalid authorization code"

    assert record["client_id"] == client_id
    assert record["redirect_uri"] == redirect_uri

    # PKCE check
    
    # print({
    #         "code_verifier" : code_verifier,
    #         "record[code_challenge]" : record["code_challenge"]  (for debugging)
    #         })
    
    assert s256(code_verifier) == record["code_challenge"], "PKCE failed"

    token = secrets.token_urlsafe(24)
    ISSUED_TOKENS[token] = {
        "client_id": client_id,
        "scope": record["scope"]
    }

    return token

In [14]:
# Full happy-path simulation (Code)

# Client-side values
CLIENT_ID = "client-123"
REDIRECT_URI = "https://agent.example.com/callback"
SCOPE = "research.read"

# PKCE setup
code_verifier = "random-secret"
code_challenge = s256(code_verifier)

# Step 1: authorize
auth_code = authorize_request(
    CLIENT_ID,
    REDIRECT_URI,
    SCOPE,
    code_challenge
)

# Step 2: token
access_token = token_request(
    auth_code,
    CLIENT_ID,
    REDIRECT_URI,
    code_verifier
)

access_token


{'code_verifier': 'random-secret', 'record[code_challenge]': 'sk5IUKMvL6Zt0dxaeaTC182dE4poZyEJPWnXs8i-0OE'}


'5pdUFbmNmPk3NHwThG3uKFLKF2pDxQ93'

In [ ]:
# wrong examples (will yeild assertion errors)

authorize_request(
    CLIENT_ID,
    "https://evil.example.com/callback",
    SCOPE,
    code_challenge
)


AssertionError: redirect_uri mismatched

In [17]:
bad_verifier = "attacker-secret"

token_request(
    auth_code,
    CLIENT_ID,
    REDIRECT_URI,
    bad_verifier
)


AssertionError: invalid authorization code